In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sys,os
from tool.att import Attention
from keras.models import load_model
import esm
import torch
from tqdm import tqdm
import requests
import logging
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [44]:
# Set gpu
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

Using device: cuda


# data load

In [45]:
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'

In [46]:
# Get data from fasta 
url = "https://rest.uniprot.org/uniprotkb/P08499.fasta"

response = requests.get(url)

if response.status_code == 200:
    fasta_data = response.text
else:
    fasta_data = "Failed to download data. Status code: " + str(response.status_code)
    
header, *sequence_lines = fasta_data.split('\n')
uniprot_id = header.split('|')[1]
sequence = ''.join(sequence_lines)

dataset = pd.DataFrame({'uniprot_id': [uniprot_id], 'seq': [sequence]})

# embedding

In [47]:
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()
model = model.to(device)

In [48]:
# Esm2 embedding
def get_rep_seq(sequences):

    batch_labels, batch_strs, batch_tokens = batch_converter(sequences)
    batch_tokens = batch_tokens.to(device)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=False)
    token_representations = results["representations"][33]
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
       
    np_list = []

    for i, ten in enumerate(sequence_representations):
        ten=ten.cpu().detach().numpy()
        np_list.append(ten)
    res = pd.DataFrame(np_list)
    res.columns = ['f'+str(i) for i in range (0,res.shape[1])]
    return res

In [49]:
df_data = list(zip(dataset.uniprot_id.index,dataset.seq))

# Run in batches
stride =2
num_iterations = len(df_data) // stride
if len(df_data) % stride != 0:
    num_iterations += 1
    
# Embedding
all_results = pd.DataFrame()

for i in tqdm(range(num_iterations)):
    
    start = i * stride
    end = start + stride

    current_data = df_data[start:end]

    rep33 = get_rep_seq(sequences=current_data)
    rep33['uniprot_id'] = dataset[start:end].uniprot_id.tolist()
    cols = list(rep33.columns)
    cols = [cols[-1]] + cols[:-1]
    rep33 = rep33[cols]
    all_results = pd.concat([all_results, rep33], ignore_index=True)

100%|██████████| 1/1 [00:00<00:00, 10.92it/s]


# predict

In [50]:
# Deepsub
model = load_model("./model/deepsub_new.h5",custom_objects={"Attention": Attention},compile=False)
predicted = model.predict(np.array(all_results.iloc[:,1:]).reshape(all_results.shape[0],1,-1))
predicted_labels = np.argmax(predicted, axis=1)
label_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 12}
y_test_transformed = [label_map[x] for x in predicted_labels]
print("These are the predicted labels:")
print(y_test_transformed)

1/1 [==============================] - 1s 596ms/step
These are the predicted labels:
[4]


In [51]:
#queen 
# Load model
model_location = "./model/QUEEN_MLPmodel_final.pkl"
with open(model_location, "rb") as f:
  QUEEN_model = pickle.load(f)
  
# Queen pred
y_test = QUEEN_model.predict(np.array(all_results.iloc[:,1:]))
inv_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 12, 10: 14, 11: 24}

y_test_transformed = np.array([inv_map[x] for x in y_test])
print("These are the predicted labels:")
print(y_test_transformed)

These are the predicted labels:
[2]


/home/linjw/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.0.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/linjw/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.0.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [52]:
#knn
with open('./model/ml_xg.pkl', 'rb') as file:
    xg_model = pickle.load(file)
xg_predict = xg_model.predict(all_results.iloc[:,1:])

inv_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 12, 10: 14, 11: 24}

y_test_transformed = np.array([inv_map[x] for x in knn_predict])
print("xg These are the predicted labels:")
print(y_test_transformed)


with open('./model/ml_knn.pkl', 'rb') as file:
    knn_model = pickle.load(file)
xg_predict = xg_model.predict(all_results.iloc[:,1:])
knn_predict = knn_model.predict(all_results.iloc[:,1:])

inv_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 12, 10: 14, 11: 24}

y_test_transformed = np.array([inv_map[x] for x in knn_predict])
print("knn hese are the predicted labels:")
print(y_test_transformed)

xg These are the predicted labels:
[4]


/home/linjw/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


knn hese are the predicted labels:
[4]


In [53]:
dataset

,uniprot_id,seq
0,P08499,MTSASAPSFNPGKGPGSAVGIALLGFGTVGTEVMRLMTEYGDELAH...


In [54]:
#blast
from tool import config as cfg
from tool.blast import getblast

data = pd.read_csv(cfg.DATA_PATH)
data = data.rename(columns={'Entry':'id','Sequence':'seq'})
dataset = dataset.rename(columns={'uniprot_id':'id'})

diamond_task = getblast(train = data, test = dataset)
diamond_task

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd --quiet


diamond v0.9.30.131 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org



diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 -e 1e-5 --quiet


diamond v0.9.30.131 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org



,id,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,P08499,Q5F8J4,39.9,436,243,7,19,443,4,431,1.700000e-74,278.9


In [55]:
data[data.id=='Q5F8J4']

,id,seq,label,organism,EC number
20459,Q5F8J4,MKPVNIGLLGLGTVGGGAAAVLRDNAEEISRRLGREIRISAMCDLS...,4,Neisseria gonorrhoeae (strain ATCC 700825 / FA...,1.1.1.3
